<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0704/07_other_callback_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Callbacks API 정리

### 지금까지 다뤄본 callback class  
1) learning 관련
- Early Stopping  
- CheckPoint
- ReduceLROnPlateau  
- LearningRateScheduler

2) logging 관련
- TensorBoard

In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10)
            ])

    return model

In [ ]:
model = create_model()

model.compile(optimizer='adam',
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2),
        tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{loss:.2f}.h5'),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.001)
    ]

In [ ]:
model.fit(x=x_train,
          y=y_train,
          epochs=3,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

Epoch 1/3
1875/1875 [==============================] - 15s 7ms/step - loss: 2.7193 - accuracy: 0.8749 - val_loss: 0.3394 - val_accuracy: 0.9301 - lr: 0.0010
Epoch 2/3
1875/1875 [==============================] - 14s 7ms/step - loss: 0.4355 - accuracy: 0.9059 - val_loss: 0.3839 - val_accuracy: 0.9220 - lr: 0.0010
Epoch 3/3
1875/1875 [==============================] - 13s 7ms/step - loss: 0.4036 - accuracy: 0.9118 - val_loss: 0.3263 - val_accuracy: 0.9359 - lr: 0.0010


## Other callbacks Summary  
1) learning 관련
- TerminateOnNaN

2) logging 관련
- BaseLogger
- CSVLogger
- ProgbarLogger
- RemoteMonitor
- History

In [ ]:
#  TerminateOnNaN : loss가 NaN이 뜨면 학습 중단

In [ ]:
# BaseLogger : epoch마다 metric의 평균을 축적하는 콜백
# CSVLogger : 결과를 CSV 파일로 스트리밍하는 콜백
# ProgbarLogger : metric을 stdout으로 프린트하는 콜백
# RemoteMonitor : 이벤트를 서버로 스트리밍하는 콜백
# History : History object로 이벤트를 기록하는 콜백

## Your Own Callback


In [ ]:
import tensorflow as tf
from tensorflow import keras

다음 세가지 메서드에 콜백을 전달할 수 있다.

- model.fit()
- model.evaluate()
- model.predict()

다음과 같이 정해진 시점마다 콜백을 수행하도록 구현할 수 있다.
> fit / evaluate / predict가 시작하거나 끝날 때  
      - on_train_begin / on_train_end  
      - on_test_begin / on_test_end  
      - on_predict_begin / on_predict_end  

   > 각 에포크가 시작하거나 끝날 때  
      - on_epoch_begin / on_epoch_end  

   > 각 배치가 시작하거나 끝날 때  
      - on_train_batch_begin / on_train_batch_end  
      - on_test_batch_begin / on_test_batch_end  
      - on_predict_batch_begin / on_predict_batch_end

In [ ]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(10)
            ])

    return model

In [ ]:
model = create_model()

model.compile(optimizer='adam',
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


## 연습하기

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("학습 시작; log 목록: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("학습 종료; log 목록: {}".format(keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("테스트 시작; log 목록: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("테스트 종료; log 목록: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("예측 시작; log 목록: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("예측 종료; log 목록: {}".format(keys))

In [ ]:
callback = CustomCallback()

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=callback,
)

학습 시작; log 목록: []
테스트 시작; log 목록: []
테스트 종료; log 목록: ['loss', 'accuracy']
학습 종료; log 목록: ['loss', 'accuracy', 'val_loss', 'val_accuracy']


In [ ]:
result = model.evaluate(x_test, y_test, batch_size=128, verbose=0, callbacks=callback)

테스트 시작; log 목록: []
테스트 종료; log 목록: ['loss', 'accuracy']


In [ ]:
pred = model.predict(x_test, batch_size=128, callbacks=callback)

예측 시작; log 목록: []
예측 종료; log 목록: []


#### loss 출력 콜백

In [ ]:
class LossPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, loss : {:7.2f}.".format(batch, logs["loss"])
        )

    def on_test_batch_end(self, batch, logs=None):
        print(
            "Up to batch {}, loss : {:7.2f}.".format(batch, logs["loss"])
        )

    def on_epoch_end(self, epoch, logs=None):
        print("loss for epoch {} : {:7.2f} ".format(epoch, logs["loss"]))

In [ ]:
model = create_model()

model.compile(optimizer='adam',
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=2,
    verbose=0,
    callbacks=[LossPrintingCallback()],
)

res = model.evaluate(
    x_test,
    y_test,
    batch_size=128,
    verbose=0,
    callbacks=[LossPrintingCallback()],
)

Up to batch 0, loss :  136.11.
Up to batch 1, loss :  105.75.
Up to batch 2, loss :   99.95.
Up to batch 3, loss :   92.79.
Up to batch 4, loss :   84.07.
Up to batch 5, loss :   76.66.
Up to batch 6, loss :   70.01.
Up to batch 7, loss :   65.33.
Up to batch 8, loss :   60.31.
Up to batch 9, loss :   56.67.
Up to batch 10, loss :   53.88.
Up to batch 11, loss :   50.86.
Up to batch 12, loss :   48.51.
Up to batch 13, loss :   46.59.
Up to batch 14, loss :   44.42.
Up to batch 15, loss :   42.42.
Up to batch 16, loss :   40.88.
Up to batch 17, loss :   39.46.
Up to batch 18, loss :   38.19.
Up to batch 19, loss :   37.10.
Up to batch 20, loss :   35.93.
Up to batch 21, loss :   35.18.
Up to batch 22, loss :   34.06.
Up to batch 23, loss :   33.33.
Up to batch 24, loss :   32.62.
Up to batch 25, loss :   31.61.
Up to batch 26, loss :   30.71.
Up to batch 27, loss :   30.01.
Up to batch 28, loss :   29.31.
Up to batch 29, loss :   28.95.
Up to batch 30, loss :   28.46.
Up to batch 31, lo

#### tf.keras.callbacks.EarlyStopping보다 나은 조기종료 콜백

In [ ]:
class 클래스명():


In [ ]:
마린 = 클래스명() 호출

In [ ]:
import numpy as np


class EarlyStoppingAtMinLoss(keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

    def __init__(self, patience=0):
        super(EarlyStoppingAtMinLoss, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get("loss")
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))


model = create_model()

model.compile(optimizer='adam',
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[LossPrintingCallback(), EarlyStoppingAtMinLoss()],
)

## LambdaCallback 이용하기

In [ ]:
tf.keras.callbacks.LambdaCallback(
    on_epoch_begin=None,
    on_epoch_end=None,
    on_batch_begin=None,
    on_batch_end=None,
    on_train_begin=None,
    on_train_end=None
)

#### arguments 정리
on_epoch_begin and on_epoch_end : epoch, logs  
on_batch_begin and on_batch_end : batch, logs  
on_train_begin and on_train_end : logs

In [ ]:
import json

In [ ]:
# 배치 시작마다 배치 출력
batch_print_callback = tf.keras.callbacks.LambdaCallback(
    on_batch_begin=lambda batch,logs: print(batch))
# lambda 입력값, 출력값

# loss json log 파일 생성
json_log = open('loss_log.json', mode='wt', buffering=1)
json_logging_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: json_log.write(
        json.dumps({'epoch': epoch, 'loss': logs['loss']}) + '\n'),
    on_train_end=lambda logs: json_log.close()
)

model = create_model()

model.compile(optimizer='adam',
            loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.fit(
    x_train,
    y_train,
    batch_size=64,
    steps_per_epoch=5,
    epochs=30,
    verbose=0,
    callbacks=[batch_print_callback,
                json_logging_callback])

0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
